In [1]:
# Install required packages
!pip install pyngrok fastapi nest-asyncio pyngrok uvicorn ollama  --quiet

# Ensure Ollama is installed
import subprocess
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)

# Start Ollama service
subprocess.Popen("OLLAMA_KEEP_ALIVE=-1 OLLAMA_NUM_PARALLEL=4 ollama serve", shell=True)
!ollama pull qwen2.5:14b

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 2049f5674b1e...   0% ▕                ▏    0 B/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   0% ▕                ▏  58 KB/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   0% ▕                ▏  15 MB/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   1% ▕                ▏  71 MB/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   2% ▕                ▏ 143 MB/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   2% ▕                ▏ 180 MB/9.0 GB                  pulling manifest 
pulling 2049f5674b1e...   3% ▕   

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from ollama import chat, ChatResponse
from typing import Optional
import json


# Set up ngrok
ngrok.set_auth_token("2kh") # replace this key with your ngrok key to make it work.

# Sample data for accounts and transactions
ACCOUNTS = {
    "12345": {
        "holder_name": "Ali Asim",
        "balance": 5000.00,
        "currency": "USD"
    },
    "67890": {
        "holder_name": "Jhanzaib",
        "balance": 7500.00,
        "currency": "USD"
    }
}

TRANSACTIONS = {
    "12345": [
        {"date": "2024-01-01", "description": "Grocery Store", "amount": -50.00, "type": "debit"},
        {"date": "2024-01-02", "description": "Restaurant", "amount": -30.00, "type": "debit"},
        {"date": "2024-01-05", "description": "Coffee Shop", "amount": -5.00, "type": "debit"},
        {"date": "2024-01-07", "description": "Bookstore", "amount": -20.00, "type": "debit"},
        {"date": "2024-01-10", "description": "Online Shopping", "amount": -100.00, "type": "debit"}
    ],
    "67890": [
        {"date": "2024-01-01", "description": "Gas Station", "amount": -40.00, "type": "debit"},
        {"date": "2024-01-02", "description": "Online Shopping", "amount": -75.00, "type": "debit"},
        {"date": "2024-01-05", "description": "Cafe", "amount": -15.00, "type": "debit"},
        {"date": "2024-01-06", "description": "Electronics Store", "amount": -200.00, "type": "debit"},
        {"date": "2024-01-09", "description": "Grocery Store", "amount": -80.00, "type": "debit"}
    ]
}

# Define Pydantic models for FastAPI
class UserInput(BaseModel):
    message: str

class UserLogin(BaseModel):
    username: str
    account_number: str

# Global session variables
current_user = None
user_account = None

# Functions for checking balance and transactions
def account_balance() -> str:
    """Gets the account balance for the current user"""
    global user_account
    if not user_account:
        return json.dumps({"message": "No active user account. Please login first."})
    
    if user_account in ACCOUNTS:
        account_data = ACCOUNTS[user_account]
        balance = account_data['balance']
        currency = account_data['currency']
        return json.dumps({"message": f"Your current balance is {currency}{balance:.2f}"})
    else:
        return json.dumps({"message": "Account not found."})

def get_transactions(num_transactions: Optional[int] = None) -> str:
    """Gets recent transactions for the current user"""
    global user_account
    if not user_account:
        return json.dumps({"message": "No active user account. Please login first."})
    
    if user_account in TRANSACTIONS:
        user_transactions = TRANSACTIONS[user_account]
        if num_transactions:
            user_transactions = user_transactions[:num_transactions]
        return json.dumps({
            "message": "Transactions fetched successfully",
            "data": user_transactions
        })
    else:
        return json.dumps({"message": "No transactions found."})

# Initialize FastAPI app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.post("/login")
def login(user: UserLogin):
    global current_user, user_account
    
    if user.account_number not in ACCOUNTS:
        raise HTTPException(status_code=404, detail="Account number not found")
        
    account_data = ACCOUNTS[user.account_number]
    if account_data['holder_name'].lower() != user.username.lower():
        raise HTTPException(status_code=401, detail="Username does not match account holder")
        
    current_user = user.username
    user_account = user.account_number
    
    return {
        "message": f"Welcome {current_user}!",
        "account_holder": account_data['holder_name'],
        "balance": account_data['balance']
    }

@app.post("/chat")
async def chat_endpoint(user_input: UserInput):
    if not user_account:
        raise HTTPException(status_code=401, detail="Please login first")
    
    prompt = user_input.message

    # Define the system prompt
    system_prompt = """
    You are a helpful banking assistant. You can:
    1. Check account balances using account_balance()
    2. View transactions using get_transactions(num_transactions: Optional[int])
    
    Always analyze the user's request carefully and use the appropriate function.
    For transaction requests, try to extract the number of transactions requested.
    Be polite and professional in your responses.
    """

    # Define available functions to be used by Ollama model
    available_functions = {
        'account_balance': account_balance,
        'get_transactions': get_transactions
    }

    # Chat with Ollama model, passing the system prompt
    response: ChatResponse = chat(
        model='qwen2.5:14b',  # Make sure you have the correct model name
        messages=[{'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': prompt}],
        tools=[account_balance, get_transactions]
    )

    if response.message.tool_calls:
        # There may be multiple tool calls in the response
        for tool in response.message.tool_calls:
            # Ensure the function is available, and then call it
            if function_to_call := available_functions.get(tool.function.name):
                function_result = function_to_call(**tool.function.arguments)
                return {"response": function_result}
            else:
                return {"response": "Function not found."}
    else:
        return {"response": "No tool call found."}

# Run the FastAPI app with ngrok setup
if __name__ == "__main__":
    
    nest_asyncio.apply()    
    # Set up ngrok tunnel
    port = 8001
    print(f"Starting server on port {port}...")
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    
    # Run FastAPI app
    uvicorn.run(app, host="0.0.0.0", port=port)


Starting server on port 8001...


INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


Public URL: https://46fb-34-127-31-109.ngrok-free.app
INFO:     203.99.191.114:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "POST /login HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     203.99.191.114:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [31]
